In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as ex

In [2]:
df = pd.read_csv("Data\\big_mart_data.csv")

In [3]:
df.shape

(8523, 12)

In [4]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [5]:
def data_info(df):
    output = []

    for col in df.columns:

        null  = np.sum(pd.isna(df[col]))
        unique = df[col].nunique()
        colType = df[col].dtype
        mode = df[col].mode().iloc[0]

        output.append([col, null, unique, colType, mode])

    output_df = pd.DataFrame(output, columns = ['column','null values', 'unique', 'dtype', 'mode'])

    describe_df = df.describe().transpose() 
    combine_df = pd.concat([ output_df.set_index('column'), describe_df], axis = 1, join='outer')

    return combine_df

In [6]:
data_info(df)

,null values,unique,dtype,mode,count,mean,std,min,25%,50%,75%,max
Item_Identifier,0,1559,object,FDG33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Item_Weight,1463,415,float64,12.15,7060.0,12.857645,4.643456,4.555,8.773750,12.600000,16.850000,21.350000
Item_Fat_Content,0,5,object,Low Fat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Item_Visibility,0,7880,float64,0.0,8523.0,0.066132,0.051598,0.000,0.026989,0.053931,0.094585,0.328391
Item_Type,0,16,object,Fruits and Vegetables,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Item_MRP,0,5938,float64,172.0422,8523.0,140.992782,62.275067,31.290,93.826500,143.012800,185.643700,266.888400
Outlet_Identifier,0,10,object,OUT027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Outlet_Establishment_Year,0,9,int64,1985,8523.0,1997.831867,8.371760,1985.000,1987.000000,1999.000000,2004.000000,2009.000000
Outlet_Size,2410,3,object,Medium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Outlet_Location_Type,0,3,object,Tier 3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Identifying the primary key of the dataset

In [7]:
df['primary key'] = df['Outlet_Identifier'] + df['Item_Identifier']
len(df['primary key'])

8523

#### Conclusions: The primary key of the data is a combination of the Outlet identifier and the item identifer. This means that the data is at a outlet identifer and item identifier level.

#### Imputing the null values

##### 1. Imputing null values in the Item_Weights column

In [8]:
# nulls = df[df['Item_Weight'].isnull()]['Item_Identifier'].unique()

# for col in nulls:
#     mode_weight = df[df['Item_Identifier'] == col]['Item_Weight'].mode()
#     df.loc[(df['Item_Identifier'] == col) & (df['Item_Weight'].isna()), 'Item_Weight'] = mode_weight
item_weight_dict = df.groupby('Item_Identifier')['Item_Weight'].apply(lambda x: x.mode().iloc[0] if not x.mode().empty else None).to_dict()

# Function to impute missing Item_Weight values
def impute_weight(row):
    if pd.isnull(row['Item_Weight']):
        return item_weight_dict[row['Item_Identifier']]
    else:
        return row['Item_Weight']

# Apply the function to the DataFrame
df['Item_Weight'] = df.apply(impute_weight, axis=1)

# Verify the changes
# print(df[df['Item_Weight'].isnull()])

In [9]:
# nulls_two = df[df['Item_Weight'].isnull()]['Item_Identifier'].unique()
# nulls_two
df['Item_Weight'].isnull().sum()

np.int64(4)

##### Conclusion: The null values have been imported. The strategy used was to find other rows where the same product was stored and take the mode value of weights for that product and impute it in the row that was missing. There are still 4 products that have no item weight as they occur only once in the entire dataset.

##### 2. Imputing null values in the Outlet Size column

# OLD OUTLET SIZE IMPUTATION CODE

In [10]:
df['Outlet_Size'].fillna("Small", inplace=True)

C:\Users\Userblg3\AppData\Local\Temp\ipykernel_3056\3831512747.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Outlet_Size'].fillna("Small", inplace=True)


In [11]:
df['Outlet_Size'].unique()

array(['Medium', 'Small', 'High'], dtype=object)

In [12]:
df['Outlet_Size']

0       Medium
1       Medium
2       Medium
3        Small
4         High
         ...  
8518      High
8519     Small
8520     Small
8521    Medium
8522     Small
Name: Outlet_Size, Length: 8523, dtype: object

In [13]:
data_info(df)

,null values,unique,dtype,mode,count,mean,std,min,25%,50%,75%,max
Item_Identifier,0,1559,object,FDG33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Item_Weight,4,415,float64,12.15,8519.0,12.875420,4.646098,4.555,8.785000,12.650000,16.850000,21.350000
Item_Fat_Content,0,5,object,Low Fat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Item_Visibility,0,7880,float64,0.0,8523.0,0.066132,0.051598,0.000,0.026989,0.053931,0.094585,0.328391
Item_Type,0,16,object,Fruits and Vegetables,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Item_MRP,0,5938,float64,172.0422,8523.0,140.992782,62.275067,31.290,93.826500,143.012800,185.643700,266.888400
Outlet_Identifier,0,10,object,OUT027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Outlet_Establishment_Year,0,9,int64,1985,8523.0,1997.831867,8.371760,1985.000,1987.000000,1999.000000,2004.000000,2009.000000
Outlet_Size,0,3,object,Small,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Outlet_Location_Type,0,3,object,Tier 3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Conclusion: We have imputed the missing values in the Outlet_Size column.

##### VISIBILITY VARIABLE IMPUTATION

In [14]:
utem = df[df['Item_Visibility'] == 0]['Item_Type'].unique()

In [15]:
for ty in utem:
    visibility_mean = df[df['Item_Type'] == ty]['Item_Visibility'].mean()
    df.loc[(df['Item_Type'] == ty) & (df['Item_Visibility'] == 0), 'Item_Visibility'] = visibility_mean

In [16]:
df['Item_Visibility'].min()

np.float64(0.003574698)

##### Conclusion: We have replaced the 0 values in item visibility with mean of visibility of that item type.

In [17]:
data_info(df)

,null values,unique,dtype,mode,count,mean,std,min,25%,50%,75%,max
Item_Identifier,0,1559,object,FDG33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Item_Weight,4,415,float64,12.15,8519.0,12.875420,4.646098,4.555000,8.785000,12.650000,16.850000,21.350000
Item_Fat_Content,0,5,object,Low Fat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Item_Visibility,0,7895,float64,0.068513,8523.0,0.070208,0.048754,0.003575,0.033085,0.061322,0.094585,0.328391
Item_Type,0,16,object,Fruits and Vegetables,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Item_MRP,0,5938,float64,172.0422,8523.0,140.992782,62.275067,31.290000,93.826500,143.012800,185.643700,266.888400
Outlet_Identifier,0,10,object,OUT027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Outlet_Establishment_Year,0,9,int64,1985,8523.0,1997.831867,8.371760,1985.000000,1987.000000,1999.000000,2004.000000,2009.000000
Outlet_Size,0,3,object,Small,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Outlet_Location_Type,0,3,object,Tier 3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Cleaning the data for Item_fat_content column

In [18]:
df['Item_Fat_Content'].unique()

array(['Low Fat', 'Regular', 'low fat', 'LF', 'reg'], dtype=object)

In [19]:
df.loc[df['Item_Fat_Content'].isin(['Low Fat', 'LF', 'low fat'])]

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,primary key
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,OUT049FDA15
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,OUT049FDN15
4,NCD19,8.930,Low Fat,0.061322,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,OUT013NCD19
7,FDP10,19.000,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636,OUT027FDP10
10,FDY07,11.800,Low Fat,0.068513,Fruits and Vegetables,45.5402,OUT049,1999,Medium,Tier 1,Supermarket Type1,1516.0266,OUT049FDY07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8515,FDH24,20.700,Low Fat,0.021518,Baking Goods,157.5288,OUT018,2009,Medium,Tier 3,Supermarket Type2,1571.2880,OUT018FDH24
8516,NCJ19,18.600,Low Fat,0.118661,Others,58.7588,OUT018,2009,Medium,Tier 3,Supermarket Type2,858.8820,OUT018NCJ19
8518,FDF22,6.865,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,Supermarket Type1,2778.3834,OUT013FDF22
8520,NCJ29,10.600,Low Fat,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1,1193.1136,OUT035NCJ29


In [20]:
df['Item_Fat_Content'] = df['Item_Fat_Content'].map({'Low Fat': 'LF', 'low fat': 'LF', 'Regular':'reg', 'LF':'LF', 'reg':'reg'})
df['Item_Fat_Content']
len(df['Item_Fat_Content'])

8523

In [21]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,primary key
0,FDA15,9.30,LF,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,OUT049FDA15
1,DRC01,5.92,reg,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,OUT018DRC01
2,FDN15,17.50,LF,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,OUT049FDN15
3,FDX07,19.20,reg,0.068513,Fruits and Vegetables,182.0950,OUT010,1998,Small,Tier 3,Grocery Store,732.3800,OUT010FDX07
4,NCD19,8.93,LF,0.061322,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,OUT013NCD19


##### Conclusion: We have cleaned the data in the item_fat_content column to represent only 2 values LF and reg

In [22]:
# df['Item_Weight'].isnull().sum()
data_info(df)

,null values,unique,dtype,mode,count,mean,std,min,25%,50%,75%,max
Item_Identifier,0,1559,object,FDG33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Item_Weight,4,415,float64,12.15,8519.0,12.875420,4.646098,4.555000,8.785000,12.650000,16.850000,21.350000
Item_Fat_Content,0,2,object,LF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Item_Visibility,0,7895,float64,0.068513,8523.0,0.070208,0.048754,0.003575,0.033085,0.061322,0.094585,0.328391
Item_Type,0,16,object,Fruits and Vegetables,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Item_MRP,0,5938,float64,172.0422,8523.0,140.992782,62.275067,31.290000,93.826500,143.012800,185.643700,266.888400
Outlet_Identifier,0,10,object,OUT027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Outlet_Establishment_Year,0,9,int64,1985,8523.0,1997.831867,8.371760,1985.000000,1987.000000,1999.000000,2004.000000,2009.000000
Outlet_Size,0,3,object,Small,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Outlet_Location_Type,0,3,object,Tier 3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Detecting outliers in the numerical data

1. Item weights

In [23]:
ex.box(data_frame=df, y=df['Item_Weight'], title="Box plot of item weights")


Conclusion: 
- The item weight column doesn't have any outliers as seen in the boxplot above. 
- The minimum and maximum value of all the items are 4.555 and 21.35 units respectively. 
- Up to 25% of the data ranges from 4.555-8.785 units. 
- The median value is 12.65 units.
- The third quartile value is 16.85 units. This means that 75% of the data lies below this value.

2. Item MRP

In [24]:
ex.box(data_frame=df, y=df['Item_MRP'], title="Box plot of Item MRP")

Conclusion:
 - The item MRP ranges from 31.29 units to 266.9 units.
 - The price of 25% of the products lies below 93.82 units.
 - The median value of item MRP is 143.0128 units
 - Quartile 3 has a value of 185.65 units. 

3. Item Visibility (Before cleaning)

In [25]:
ex.box(data_frame=df, y=df['Item_Visibility'], title="Box plot of Item Visibility", color=df['Outlet_Type'])


Conclusion:
 - Item visibility seems to have quite a few outliers above the upper whisker of the distribution.
 - Minimum item visibility is 0% (Needs to be cleaned).
 - Median value for item visibility is 5.4%
 - 75% of the data has an item visibility of 9.5% or below.
 - The upper fence of this variable is 19.57%, however the maximum value for item visibility happens to be 32.83%

In [26]:
ex.histogram(data_frame=df,x=df['Item_Visibility'], title="Histogram of item visibility", color=df['Outlet_Type'])

#### Checking for any duplicate rows in the dataset

In [27]:
df.duplicated().unique()

array([False])

#### Conclusion: There are no duplicate values in the dataset.

#### Dropping the null item_weight rows. Since they are only a small proportion of null values we have decided to ignore them.

In [28]:
df.shape

(8523, 13)

In [29]:
df = df.dropna()

In [30]:
df.shape

(8519, 13)

In [31]:
df

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,primary key
0,FDA15,9.300,LF,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,OUT049FDA15
1,DRC01,5.920,reg,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,OUT018DRC01
2,FDN15,17.500,LF,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,OUT049FDN15
3,FDX07,19.200,reg,0.068513,Fruits and Vegetables,182.0950,OUT010,1998,Small,Tier 3,Grocery Store,732.3800,OUT010FDX07
4,NCD19,8.930,LF,0.061322,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,OUT013NCD19
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8518,FDF22,6.865,LF,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,Supermarket Type1,2778.3834,OUT013FDF22
8519,FDS36,8.380,reg,0.046982,Baking Goods,108.1570,OUT045,2002,Small,Tier 2,Supermarket Type1,549.2850,OUT045FDS36
8520,NCJ29,10.600,LF,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1,1193.1136,OUT035NCJ29
8521,FDN46,7.210,reg,0.145221,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3,Supermarket Type2,1845.5976,OUT018FDN46


In [32]:
data_info(df)

,null values,unique,dtype,mode,count,mean,std,min,25%,50%,75%,max
Item_Identifier,0,1555,object,FDG33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Item_Weight,0,415,float64,12.15,8519.0,12.875420,4.646098,4.555000,8.785000,12.650000,16.850000,21.350000
Item_Fat_Content,0,2,object,LF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Item_Visibility,0,7891,float64,0.068513,8519.0,0.070190,0.048741,0.003575,0.033085,0.061322,0.094558,0.328391
Item_Type,0,16,object,Fruits and Vegetables,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Item_MRP,0,5936,float64,172.0422,8519.0,141.010019,62.283594,31.290000,93.844900,143.047000,185.676600,266.888400
Outlet_Identifier,0,10,object,OUT013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Outlet_Establishment_Year,0,9,int64,1985,8519.0,1997.837892,8.369105,1985.000000,1987.000000,1999.000000,2004.000000,2009.000000
Outlet_Size,0,3,object,Small,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Outlet_Location_Type,0,3,object,Tier 3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
